# Introduction to AVISO-data

We will use the satellite-altimetry-derived ocean-surface current product AVISO-data as a dataset to validate the methods. The data is freely available from the [AVISO](https://data.marine.copernicus.eu/products).

The sea surface height $ h $ acts as a stream function for the geostrophic ocean surface velocity field. Particle trajectories on the ocean surface are approximately trajectories of the 2D system of ODEs:

\begin{align}
U &= -\dfrac{g}{Rf(\theta)}\partial_{\theta}h(\phi, \theta, t) \tag{1} \\
V &= \dfrac{g}{Rf(\theta)}\partial_{\varphi}h(\phi, \theta, t) \tag{2},
\end{align}

where $ \varphi, \theta $ respectively are the longitudinal and latitudinal position of the particle, $ g $ is the constant of gravity, $ R $ is the mean radius, $ f(\theta) = 2\Omega \sin(\theta) $ is the coriolis parameter and $ \Omega $ is the mean angular velocity of the earth. $ \theta_0 $ is a reference value generally taken to be the averaged $ \theta $ in the considered spatial domain. U, V are the zonal and meridional velocity field in units of meters/second. We then need to convert it to deg/day using the function [convert_meters_per_second_to_deg_per_day](convert_meters_per_second_to_deg_per_day.ipynb).

# Add Folders to Path

We start by adding the necessary folders to the current working path.

In [1]:
# import sys, os
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-2])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

# add integration folder to current working path
sys.path.append(parent_directory+"/subfunctions/integration")

# Import data

In [8]:
from netCDF4 import Dataset, num2date
from datetime import datetime
import numpy as np

from scipy.interpolate import RectBivariateSpline as RBS

from tqdm.notebook import tqdm

# import function to convert meters/second to deg/day
from ipynb.fs.defs.convert_meters_per_second_to_deg_per_day import *

from math import cos, pi

def SSH_to_vel(file):
    '''
    INPUT:
        file: '.nc'-file downloaded from Copernicus. This file contains the SSH data
    
    OUTPUT:
        x: longitude in degrees, array(1, NX)
        y: latitude in degrees, array(1, NY)
        time: time, array (1, NT)
        U: zonal velocity component in deg/days, array(NY, NX, NT)
        V: zonal velocity component, array(NY, NX, NT)
    
    '''
    
    fileobj = Dataset(file, 'r')

    # x: longitudinal coordinate (in degrees)
    x = fileobj.variables['longitude'][:]
    # y: latitudinal coordinate (in degrees)
    y = fileobj.variables['latitude'][:]
    # ssh: Sea-surface height profile (in m)
    ssh = fileobj.variables['zos'][:,:,:].transpose((1, 2, 0))
    
    # time from .nc file
    nctime = fileobj.variables['time'][:]
    t_unit = fileobj.variables['time'].units  # get unit  "days since 1950-01-01T00:00:00Z"

    t_cal = fileobj.variables['time'].calendar
    
    ###### define parameters ######
    
    NY = y.shape[0]
    NX = x.shape[0]
    
    # theta
    theta = y # degrees
    
    # earths acceleration
    g = 9.81 # m/s^2 
    
    # coriolis parameter
    f = 2*7.2921*(10**(-5))*np.sin(theta*np.pi/180)
    
    # Radius of the earth
    earthRadius = 6371*(10**3)
    
    # defined domain
    defined_domain = np.isfinite(ssh[:,:,0])
    
    # set all nans to 0 to perform interpolation
    ssh[np.isnan(ssh)] = 0

    date_gregorian = [num2date(nctime[i], units=t_unit, calendar=t_cal) for i in range(len(nctime))]
    
    # time (in days)
    time = (np.array(nctime)) - nctime[0]
    
    Phi, Theta = np.meshgrid(x, y)

    u_m, v_m = np.zeros(ssh.shape), np.zeros(ssh.shape)
    
    # compute velocities in meters using the quasi-geostrophic equation with the f-plane approximation
    for t in tqdm(range(u_m.shape[-1])):
        # compute interpolant for ssh in order to evaluate derivatives
        interpolant_ssh = RBS(y, x, ssh[:,:,t])
        
        dsshdtheta = interpolant_ssh(Theta[:,0], Phi[0,:], dy = 1)
        dsshdphi = interpolant_ssh(Theta[:,0], Phi[0,:], dx = 1)
        
        for i in range(NY):
            for j in range(NX):
                u_m[i,j,t] = -(g/f[i])*dsshdtheta[i,j]/earthRadius*180/np.pi
                v_m[i,j,t] = (g/f[i])*dsshdphi[i,j]/earthRadius*180/np.pi
        
        # set all values which originally had nan to nan again
        u_m[~defined_domain,t] = np.nan
        v_m[~defined_domain,t] = np.nan
    
    # convert meters/second to degrees/day
    X, Y = np.meshgrid(x, y)
    U, V = convert_meters_per_second_to_deg_per_day(X, Y, u_m, v_m)
            
    time =  time.reshape(1,-1)
            
    return x, y, U, V, time.reshape(1,-1)

#Import velocity data from file in data-folder
x, y, U, V, time_data = SSH_to_vel('SSH.nc')

  0%|          | 0/32 [00:00<?, ?it/s]

In [3]:
# save data
import scipy.io as sio

mdic = {"u": U, "v": V, "x": x, "y": y, "t": time_data}
sio.savemat('AVISO_from_ssh.mat', mdic)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = plt.axes()
ax.quiver(x, y, U[:,:,0], V[:,:,0])
plt.show()